In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from transformers import T5ForConditionalGeneration,T5Tokenizer

In [ ]:
informal_to_formal_pipeline = pipeline("text2text-generation", 
                                       model=AutoModelForSeq2SeqLM.from_pretrained("informal_to_formal_styletransfer"), 
                                       tokenizer=AutoTokenizer.from_pretrained("informal_to_formal_styletransfer"))

formal_to_informal_pipeline = pipeline("text2text-generation", 
                                       model=AutoModelForSeq2SeqLM.from_pretrained("formal_to_informal_styletransfer"), 
                                       tokenizer=AutoTokenizer.from_pretrained("formal_to_informal_styletransfer"))

t5_model = T5ForConditionalGeneration.from_pretrained('t5_paraphraser')
t5_tokenizer = T5Tokenizer.from_pretrained('t5_paraphraser')
def t5_pipeline(t5_model, t5_tokenizer, sentence): 
    torch.manual_seed(42)
    encoding = tokenizer.encode_plus("paraphrase: " + sentence + " </s>",  pad_to_max_length=True, return_tensors="pt")
    input_ids, attention_masks = encoding["input_ids"], encoding["attention_mask"]
    beam_outputs = model.generate(input_ids=input_ids, attention_mask=attention_masks, do_sample=True, max_length=256, top_k=120,
                                  top_p=0.98, early_stopping=True, num_return_sequences=10)
    final_outputs =[]
    for beam_output in beam_outputs:
        sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
        if sent.lower() != sentence.lower() and sent not in final_outputs:
            final_outputs.append(sent)
    return final_outputs

In [ ]:
sentence = "The best cloud on the market is Microsoft Azure."

print(f"Original Sentence: {sentence}\n")
print(f"Informal to Formal:\n\t{informal_to_formal_pipeline(sentence)[0]['generated_text']}")
print(f"Formal to Informal:\n\t{formal_to_informal_pipeline(sentence)[0]['generated_text']}")
print("T5 Options:")
for idx, sent in enumerate(t5_pipeline(t5_model, t5_tokenizer, sentence)): 
    print(f"{idx:5d}\t{sent}")

